# General Stuff

In [5]:
# import requests

# # Define the URL of your FastAPI server
# url = "http://localhost:8000"  # Update with your server's URL

# # Define the data to send as JSON
# data = {
#     "text1": "Input Text 1",
#     "text2": "Input Text 2"
# }

# # Send a POST request with JSON data
# response = requests.post(url, json=data)

# # Check the response
# if response.status_code == 200:
#     result = response.json()
#     similarity = result["similarity"]
#     print(f"Cosine Similarity: {similarity}")
# else:
#     print(f"Request failed with status code: {response.status_code}")

In [3]:
import requests

# Define the URL of your FastAPI server
url = "http://localhost:8000/add_person"  # Update with your server's URL

# Define the data to send as JSON
person_data = {
    "first_name": "enas",
    "last_name": "khwaileh",
    "age": 35
}

# Send a POST request with JSON data to add a person
response = requests.post(url, json=person_data)

# Check the response
if response.status_code == 200:
    result = response.json()
    print(result["message"])
else:
    print(f"Request failed with status code: {response.status_code}")


Person added successfully


In [2]:
import os
os.listdir("uploads")

['test.csv', 'test2.csv', 'test3.csv']

In [13]:
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd


db = np.array(['Data Scientist',' Machine Learning Engineer','Data Analyst', 'Software Developer','Front End Developer','Back End Developer','Mathematician','Physicist'])
model = SentenceTransformer("all-mpnet-base-v2")
encodings = model.encode(db)
k=5
query = 'ML Engineer'
query_emb = model.encode(query)
cos_sims = util.cos_sim(query_emb, encodings).numpy()[0]
top_k_indices = np.argsort(-cos_sims)[:k]

top_k_results = list(zip(db[top_k_indices],cos_sims[top_k_indices]))
top_k_results = pd.DataFrame(top_k_results, columns=['TableName','SimilarityScore']).to_dict()

{'TableName': {0: ' Machine Learning Engineer',
  1: 'Data Scientist',
  2: 'Data Analyst',
  3: 'Software Developer',
  4: 'Physicist'},
 'SimilarityScore': {0: 0.8346510529518127,
  1: 0.6666718125343323,
  2: 0.6132876873016357,
  3: 0.563723087310791,
  4: 0.5109235644340515}}

In [1]:
import os
from os.path import isfile, join
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from prettytable import PrettyTable


c:\Users\leandros\Documents\GitHub\mira_gui\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
datasetpath =  f"semantic_matching/data/GroundTruth_Data/"
embeddingspath = f"semantic_matching/data/newdataembedding/"

files = [f for f in os.listdir(datasetpath) if (isfile(join(datasetpath, f)) and f.endswith('.csv'))]
embeddingsFiles = [f for f in os.listdir(embeddingspath) if (isfile(join(embeddingspath, f)) and f.endswith('.pt'))]

embeddings = torch.empty(0, device = 'cpu')
for embeddingsFile in embeddingsFiles:
    embeddingsOffile = torch.load(embeddingspath+ embeddingsFile, map_location=torch.device('cpu'))
    embeddings = torch.concat([embeddings, embeddingsOffile])

In [15]:
filesContent = {}
content = []

for file in files:
    try:
        df = pd.read_csv( datasetpath + file, encoding = "ISO-8859-1", on_bad_lines = "skip")
        filesContent[file] = df.columns.tolist()
        content += df.columns.tolist()
        for row in df.values:
            filesContent[file] += list(map(str,row))
            content += list(map(str,row))
    except Exception as e:
        print(f"Error in reading a csv file: file name: {file}, Error message: {e}")

Error in reading a csv file: file name: Airbnb_Open_Data.csv, Error message: Error tokenizing data. C error: EOF inside string starting at row 36


In [5]:
os.listdir(datasetpath)

['GroundTruth_Data']

In [19]:

model = SentenceTransformer("all-mpnet-base-v2")

query_embedding1 = model.encode('Computer Scientist')
query_embedding2 = model.encode('parental level of education')
cos_sims = util.cos_sim(query_embedding1, query_embedding2).numpy()[0]
cos_sims

array([0.18616231], dtype=float32)

In [ ]:
import os
from os.path import isfile, join
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from prettytable import PrettyTable

def readFiles():

	datasetpath =  f"../data/GroundTruth_Data/"
	embeddingspath = f"../data/newdataembedding/"

	files = [f for f in os.listdir(datasetpath) if (isfile(join(datasetpath, f)) and f.endswith('.csv'))]
	embeddingsFiles = [f for f in os.listdir(embeddingspath) if (isfile(join(embeddingspath, f)) and f.endswith('.pt'))]
	
	embeddings = torch.empty(0, device = 'cpu')
	for embeddingsFile in embeddingsFiles:
		embeddingsOffile = torch.load(embeddingspath+ embeddingsFile)
		embeddings = torch.concat([embeddings, embeddingsOffile])

	
	filesContent = {}
	content = []
	
	for file in files:
		try:
			df = pd.read_csv( datasetpath + file, encoding = "ISO-8859-1", on_bad_lines = "skip")
			filesContent[file] = df.columns.tolist()
			content += df.columns.tolist()
			for row in df.values:
				filesContent[file] += list(map(str,row))
				content += list(map(str,row))
		except Exception as e:
			print(f"Error in reading a csv file: file name: {file}, Error message: {e}")
		
	return filesContent, content, embeddings

def getMeXResults(X,top_results, filesContent, content):
	XResults = []
	i = 0
	cellsFileCombi = []
	for score, idx in zip(top_results[0], top_results[1]):
		if float("{:.4f}".format(score)) >= 0.3:
			for filename, val in filesContent.items():
				if content[idx] in val:
					if (filename, content[idx]) not in cellsFileCombi:
						cellsFileCombi.append((filename, content[idx]))
						XResults.append((filename, content[idx], score))
						i += 1
						if i == X:
							return XResults
	return XResults

def results(query, filesContent, content, embeddings):   
	
	symmetric_embedder = SentenceTransformer('all-mpnet-base-v2')
	results = {}
	
	query_embedding = symmetric_embedder.encode(query, convert_to_tensor = True)
	top_k = min(10, len(embeddings))
	cos_scores = util.cos_sim(query_embedding, embeddings)[0]
	top_results = torch.topk(cos_scores, len(embeddings), sorted = True)
	
	results = getMeXResults(top_k, top_results, filesContent, content)
	return results

def matching():
	filesContent, content, embeddings = readFiles()
	while True:
		query = "Rock Music"
		result = results(query, filesContent, content, embeddings)
		print("============================")
		print(f"your query is: {query}")
		
		table = [['Filename', 'Cell Value', 'Score']]
		for i in result:
			table.append([i[0], i[1], "{:.4f}".format(i[2])])
		tab = PrettyTable(table[0])
		tab.add_rows(table[1:])
		print(tab)
		
if __name__ == "__main__":
	matching()


# Embeddings generation

In [148]:
import os
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")

In [152]:

def create_embeddings(table_name, base_path):

    if os.path.exists(base_path + table_name.replace('.csv', '.pt') ):
        return 'Embeddings file already exist'

    df = pd.read_csv(base_path + table_name)

    col_names = df.columns.tolist()
    df_columns = pd.DataFrame([col_names], columns=col_names)

    df = pd.concat([df_columns,df], ignore_index=True)
    df = df.map(lambda cell_value: str(cell_value))
    cell_values_flat = df.values.flatten()

    # Create embeddings for all cell values
    embeddings_list_flat = model.encode(cell_values_flat)

    # Save embeddings to pt file
    torch.save(embeddings_list_flat, base_path + table_name.replace('.csv', '.pt') )

    return 'Embeddings created successfully'

In [158]:
table_name = 'shopping_trends.csv'
base_path = 'semantic_matching/data/'

create_embeddings(table_name, base_path)

'Embeddings created successfully'

In [162]:
def create_data(table_name):

    print(f'Processing table: {table_name}')

    # Read csv data
    # base_path = 'semantic_matching/data/'
    df = pd.read_csv(table_name)

    # Add column names as first row
    col_names = df.columns.tolist()
    df_columns = pd.DataFrame([col_names], columns=col_names)
    df = pd.concat([df_columns,df], ignore_index=True)
    df = df.map(lambda cell_value: str(cell_value))
    cell_values_flat = df.values.flatten()

    # Create embeddings for all cell values
    # embeddings_list_flat = model.encode(cell_values_flat)
    embeddings_path = table_name.replace('.csv', '.pt')
    assert os.path.exists(embeddings_path), "Embeddings path doesn't exist"
    embeddings_list_flat = torch.load(embeddings_path)
    
    # cell_numbers_flat = list(range(len(cell_values_flat)))

    col_names_flat = col_names * int((len(cell_values_flat)/len(col_names)))

    # Final DataFrame
    name_and_embs = pd.DataFrame(list(zip([table_name.split('/')[-1]]*len(embeddings_list_flat), cell_values_flat , col_names_flat, embeddings_list_flat)), columns=['TableName','CellValue','CellValue_Column','Embeddings'])

    return name_and_embs


In [126]:
def get_top_k(query, input_df, k):

    query_emb = model.encode(query)
    cos_sims = util.cos_sim(query_emb, input_df['Embeddings']).numpy()[0]
    input_df['SimilaritiyScores'] = cos_sims
    input_df =  input_df.drop_duplicates(subset='CellValue').reset_index(drop=True)
    input_df =  input_df.sort_values('SimilaritiyScores', ascending=False)

    return input_df.iloc[:k]

In [140]:
def batch_semantic_matching(query, table_names, k):

    total_results_df = pd.DataFrame([],columns=['TableName','CellValue','CellValue_Column','Embeddings'])

    for table_name in table_names:

        # Calculate cosine similarities 
        res_df = create_data(table_name)
        total_results_df = pd.concat([total_results_df, res_df], ignore_index=True)

    # Get top-k similar results
    top_k_df = get_top_k(query, total_results_df, k).reset_index(drop=True)

    return top_k_df


In [160]:
# Get all csv files:
base_path = 'semantic_matching/data/'
csv_files = [base_path+file for file in os.listdir(base_path) if file.endswith('.csv')]
csv_files

['semantic_matching/data/jobs.csv',
 'semantic_matching/data/shopping_trends.csv']

In [161]:
csv_files[0].split('/')[-1]

'jobs.csv'

In [164]:
# table_names = ['jobs.csv', 'shopping_trends.csv']

batch_semantic_matching('Fashion Data Analyst', csv_files, 5).to_json()

Processing table: semantic_matching/data/jobs.csv
Processing table: semantic_matching/data/shopping_trends.csv


'{"TableName":{"0":"shopping_trends.csv","1":"jobs.csv","2":"jobs.csv","3":"shopping_trends.csv","4":"shopping_trends.csv"},"CellValue":{"0":"Clothing","1":"Data Analyst","2":"Data Scientist","3":"Blouse","4":"Dress"},"CellValue_Column":{"0":"Category","1":"Job Name","2":"Job Name","3":"Item Purchased","4":"Item Purchased"},"Embeddings":{"0":[0.0262115393,0.0403750688,-0.0047923974,-0.022925755,-0.0115456907,-0.0317161977,-0.0106132897,0.0288321972,-0.0355460383,0.0005748161,0.087138705,0.0225441512,0.0386847593,0.0312374271,0.061333321,-0.0613291636,-0.005983029,-0.0055520893,-0.0070899283,0.0049786558,0.0037464222,0.0347625799,0.0151813058,-0.0259783324,0.0649028271,-0.050383877,-0.0104809524,-0.0296868086,-0.0270534083,0.0216399692,0.0679312125,-0.0366258137,0.0103977527,-0.0771904662,0.0000019679,-0.0134918196,0.0263868887,-0.0281026959,0.0047026034,-0.0163880046,0.0331259631,-0.0181938205,-0.0086831041,0.0345182829,0.0160441436,-0.0570508353,0.058210358,0.0745431781,-0.0554270856,

# QDRANT

In [1]:
from qdrant_client import models
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import torch
from config import *

c:\Users\leandros\Documents\GitHub\mira_gui\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiments

In [19]:
# model = SentenceTransformer("all-mpnet-base-v2")
model = SentenceTransformer(SBERT_PATH)
# model.save('sbertmodel')

In [ ]:
# docker run -p 6333:6333 -v $pwd/qdrant_storage:/qdrant/storage:z qdrant/qdrant

In [20]:
client = QdrantClient(host="localhost", port=6333)

In [174]:
from qdrant_client.http.models import Distance, VectorParams

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [173]:
# client.delete_collection(COLLECTION_NAME)

True

In [24]:
table_name = 'semantic_matching/data/music.csv'
df = pd.read_csv(table_name)

In [25]:
# Add column names as first row
col_names = df.columns.tolist()
df_columns = pd.DataFrame([col_names], columns=col_names)
df = pd.concat([df_columns,df], ignore_index=True)
# df = df.map(lambda cell_value: str(cell_value))
cell_values_flat = df.values.flatten()

# Create embeddings for all cell values
# embeddings_list_flat = model.encode(cell_values_flat)
embeddings_path = table_name.replace('.csv', '.pt')
assert os.path.exists(embeddings_path), "Embeddings path doesn't exist"
embeddings_list_flat = torch.load(embeddings_path)

# cell_numbers_flat = list(range(len(cell_values_flat)))

col_names_flat = col_names * int((len(cell_values_flat)/len(col_names)))

# Final DataFrame
name_and_embs = pd.DataFrame(list(zip([table_name.split('/')[-1]]*len(embeddings_list_flat), cell_values_flat , col_names_flat, embeddings_list_flat)), columns=['TableName','CellValue','CellValue_Column','Embeddings'])


In [170]:
name_and_embs =  name_and_embs.drop_duplicates(subset='CellValue').reset_index(drop=True)

In [171]:
name_and_embs.head(10)

,TableName,CellValue,CellValue_Column,Embeddings
0,music.csv,Title,Title,"[0.0868767, 0.11400506, -0.0060333717, 0.05466..."
1,music.csv,Artist,Artist,"[0.020932574, 0.059109934, -0.012733743, 0.010..."
2,music.csv,Album,Album,"[0.021122865, 0.08372989, -0.008294431, -0.010..."
3,music.csv,Year,Year,"[0.002644233, 0.06685345, 0.015512964, -0.0041..."
4,music.csv,Genre,Genre,"[0.01897388, 0.04270177, 0.004695017, 0.009216..."
5,music.csv,Bohemian Rhapsody,Title,"[0.004845577, 0.043399297, -0.0050957813, -0.0..."
6,music.csv,Queen,Artist,"[0.0016129506, 0.007959083, -0.0048746886, -0...."
7,music.csv,A Night at the Opera,Album,"[0.0024867465, 0.052483708, -0.035862844, -0.0..."
8,music.csv,1975,Year,"[-0.01719815, 0.025505552, -0.0021372891, 0.02..."
9,music.csv,Rock,Genre,"[-0.027744388, 0.05039046, -0.008959857, 0.025..."


In [172]:
name_and_embs_dict = name_and_embs.to_dict('records')

In [138]:
# payload = map(lambda row: row[1].to_dict(), name_and_embs[['TableName','CellValue','CellValue_Column']].iterrows())

In [139]:
# client.upload_collection(
#     collection_name="miraculous",
#     vectors=name_and_embs['Embeddings'].values,
#     payload=payload,
#     ids=None,  # Vector ids will be assigned automatically
#     batch_size=256,  # How many vectors will be uploaded in a single request?
# )

In [28]:
import uuid

In [175]:
client.upload_records(
    collection_name=COLLECTION_NAME,
    records=[
        models.Record(
            id=str(uuid.uuid4()), vector=doc["Embeddings"].tolist(), payload={key: value for key, value in doc.items() if key != 'Embeddings'}
        )
        for idx, doc in enumerate(name_and_embs_dict)
    ],
)

In [176]:
hits = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=model.encode("rock music").tolist(),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'CellValue': 'Rock and Roll', 'CellValue_Column': 'Genre', 'TableName': 'music.csv'} score: 0.7538078
{'CellValue': 'Rock', 'CellValue_Column': 'Genre', 'TableName': 'music.csv'} score: 0.74367994
{'CellValue': 'Genre', 'CellValue_Column': 'Genre', 'TableName': 'music.csv'} score: 0.6101377


In [6]:
table_name = 'semantic_matching/data/music.csv'
df = pd.read_csv(table_name)

In [7]:
df.to_dict('records')

[{'Title': 'Bohemian Rhapsody',
  'Artist': 'Queen',
  'Album': 'A Night at the Opera',
  'Year': 1975,
  'Genre': 'Rock'},
 {'Title': 'Billie Jean',
  'Artist': 'Michael Jackson',
  'Album': 'Thriller',
  'Year': 1982,
  'Genre': 'Pop'},
 {'Title': 'Hotel California',
  'Artist': 'Eagles',
  'Album': 'Hotel California',
  'Year': 1976,
  'Genre': 'Rock'},
 {'Title': 'Like a Rolling Stone',
  'Artist': 'Bob Dylan',
  'Album': 'Highway 61 Revisited',
  'Year': 1965,
  'Genre': 'Rock'},
 {'Title': 'Stairway to Heaven',
  'Artist': 'Led Zeppelin',
  'Album': 'Led Zeppelin IV',
  'Year': 1971,
  'Genre': 'Rock'},
 {'Title': 'Imagine',
  'Artist': 'John Lennon',
  'Album': 'Imagine',
  'Year': 1971,
  'Genre': 'Pop'},
 {'Title': 'Purple Haze',
  'Artist': 'Jimi Hendrix',
  'Album': 'Are You Experienced',
  'Year': 1967,
  'Genre': 'Rock'},
 {'Title': 'I Want to Hold Your Hand',
  'Artist': 'The Beatles',
  'Album': 'Meet The Beatles!',
  'Year': 1963,
  'Genre': 'Pop'},
 {'Title': 'Boogie W

## Main usage with function

In [2]:
from semantic_matching.code.semantic_matching_tables import add_to_collection, delete_collection, create_collection, count_table_occurencies

In [3]:
client = QdrantClient(host="localhost", port=6333)
qdrant_is_running = True

In [5]:
"-" if not qdrant_is_running else "Yes" if count_table_occurencies("shopping_trends.csv", client, COLLECTION_NAME)>0 else "No"

Counting occurencies of table: shopping_trends.csv


'No'

In [5]:
delete_collection(client , COLLECTION_NAME)

Deleting collection: miraculous


In [6]:
create_collection(client , COLLECTION_NAME)

Creating collection: miraculous


In [7]:
add_to_collection(UPLOAD_DIR+"music.csv", client, COLLECTION_NAME )

QDRANT: Adding table semantic_matching/data/music.csv to miraculous collection


'Success'

In [8]:
add_to_collection(UPLOAD_DIR+"jobs.csv", client, COLLECTION_NAME )

QDRANT: Adding table semantic_matching/data/jobs.csv to miraculous collection


'Success'

In [24]:
filename = "jobs.csv"

In [25]:
result = client.count(
    collection_name=COLLECTION_NAME,
    count_filter=models.Filter(
        must=[
             models.FieldCondition(key="TableName", match=models.MatchValue(value=filename)),
        ]
    ),
    exact=True,
)

result.count

152

In [22]:
result.count

152

In [17]:
client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="TableName", match=models.MatchValue(value="jobs.csv")),
        ]
    ),
    # limit=1,
    with_payload=True,
    # with_vectors=False,
) 

([Record(id='035cd210-e1a7-4006-a058-ed455dcf5f6f', payload={'CellValue': '25-01-22', 'CellValue_Column': 'Date Posted', 'TableName': 'jobs.csv'}, vector=None),
  Record(id='036b76f4-abf7-44ec-a175-4e777fc07f11', payload={'CellValue': 'Quality Assurance Analyst', 'CellValue_Column': 'Job Name', 'TableName': 'jobs.csv'}, vector=None),
  Record(id='071f9ead-1948-4b25-9c9b-336b0f890085', payload={'CellValue': '05-05-22', 'CellValue_Column': 'Date Posted', 'TableName': 'jobs.csv'}, vector=None),
  Record(id='074b96cc-02b4-4517-83b0-957a91cf6a5a', payload={'CellValue': 115000, 'CellValue_Column': 'Salary', 'TableName': 'jobs.csv'}, vector=None),
  Record(id='0979312a-5a89-4f55-860f-77dafb30d39c', payload={'CellValue': 'Customer Success Specialist', 'CellValue_Column': 'Job Name', 'TableName': 'jobs.csv'}, vector=None),
  Record(id='0b69414a-91c9-4e0d-a1a9-96aa7e5b584c', payload={'CellValue': 95000, 'CellValue_Column': 'Salary', 'TableName': 'jobs.csv'}, vector=None),
  Record(id='0fa22209-9

## TESTING

In [46]:
import pandas as pd

In [61]:
class Table():

    def __init__(self, id, name, description, data, columns, metadata):
        self.id = id
        self.name = name
        self.description = description
        self.data = data
        self.columns = columns
        self.metadata = metadata


    # def addRow(row):


    # def addColumn(column):


    # def updateRow(row_id):


    # def deleteRow(row_id):


    # def deleteColumn(column_id):


    def display(self):
        print('id :', self.id )
        print('name :', self.name )
        print('description :', self.description)
        print('columns :', self.columns )
        print('metadata :', self.metadata )
        print('data :', pd.DataFrame(self.data, columns=self.columns))



In [62]:
my_table = Table(123414, 'MyTalbe', 'My description', [[1,2],[4,5]], ['FirstCol', 'SecondCol'], {'created_at':'today'})

In [63]:
my_table.display()

id : 123414
name : MyTalbe
description : My description
columns : ['FirstCol', 'SecondCol']
metadata : {'created_at': 'today'}
data :    FirstCol  SecondCol
0         1          2
1         4          5


In [1]:
# Function Polymorphism
len([1,2,3])
len('string')

# Class Polymorphism

class Car:
  def __init__(self, brand, model):
    self.brand = brand
    self.model = model

  def move(self):
    print("Drive!")


class Boat:
  def __init__(self, brand, model):
    self.brand = brand
    self.model = model

  def move(self):
    print("Sail!")


class Plane:
  def __init__(self, brand, model):
    self.brand = brand
    self.model = model

  def move(self):
    print("Fly!")


car1 = Car("Ford", "Mustang")       #Create a Car class
boat1 = Boat("Ibiza", "Touring 20") #Create a Boat class
plane1 = Plane("Boeing", "747")     #Create a Plane class


for x in (car1, boat1, plane1):
  x.move()




# Inheritance Class Polymorphism


class Vehicle:
  def __init__(self, brand, model):
    self.brand = brand
    self.model = model

  def move(self):
    print("Move!")

class Car(Vehicle):
  pass


class Boat(Vehicle):
  def move(self):
    print("Sail!")


Drive!
Sail!
Fly!


In [17]:
from dataclasses import dataclass

@dataclass
class Car:
    brand : str
    model : str

    def move(self):
        print(type(self.brand))

In [18]:
Car(5, 5).move()

<class 'int'>


In [25]:
from pydantic import BaseModel

class Car(BaseModel):
    brand : str
    model : str

    def move(self):
        print(type(self.brand))

In [26]:
Car(brand="5",model="arg").move()

<class 'str'>
